In [0]:
import os
import pandas as pd
import numpy as np
import timeit
import ujson
#Usamos ujson que parece que es el que hace la importación más rápida. Añadir la documentación donde se comenta.
#from google.colab import drive  #Lo pongo más abajo, porque si no me da error (por qué??!!)

In [4]:
!ls -lh

total 4.0K
drwxr-xr-x 2 root root 4.0K Oct 23 16:44 sample_data


##IMPORTACIÓN DESDE GOOGLE DRIVE

In [0]:
#Esto sólo funciona en Google Colab porque tiene una librería de Google Colab.
#Más documentación en:
#https://colab.research.google.com/notebooks/io.ipynb
#en el apartado PyDrive.
#PyDrive documentación:
#https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

In [0]:
#Estos ficheros están en mi google drive.

#Vínculo a la carpeta comprimida con todas las pelis en mi Google Drive:
#https://drive.google.com/open?id=1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL

#movies
file_id = '1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("movies.zip")

#trial
#file_id = '1ivGsdMnbPyHuxXPoZAnQVnjKp82-hq9o'

#downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile("trial.zip")

In [2]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Oct 26 15:33 adc.json
drwx------ 3 root root 4.0K Oct 26 15:38 drive
-rw-r--r-- 1 root root 1.5M Oct 26 15:46 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 1.5G Oct 26 15:33 movies.zip
drwxr-xr-x 2 root root 4.0K Oct 23 16:44 sample_data
drwxr-xr-x 3 root root 4.0K Oct 26 15:34 themoviedb_data


In [8]:
#Da un error pero no tiene consecuencias

#!mkdir themoviedb_data
#!cd themoviedb_data && unzip -q ../trial.zip
#!ls -hl themoviedb_data

!mkdir themoviedb_data
!cd themoviedb_data && unzip -q ../movies.zip
!ls -hl themoviedb_data

warning [../movies.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../movies.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #409792).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 12M
drwxr-xr-x 2 root root 12M Oct 26 15:35 movies


In [9]:
!ls -hl ./themoviedb_data/movies | head -10

total 4.3G
-rw-r--r-- 1 root root 7.4K Aug 26 18:10 100009.json
-rw-r--r-- 1 root root  22K Aug 25 12:44 10000.json
-rw-r--r-- 1 root root  24K Aug 26 18:10 100010.json
-rw-r--r-- 1 root root  18K Aug 26 18:10 100011.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100013.json
-rw-r--r-- 1 root root 6.8K Aug 26 18:10 100016.json
-rw-r--r-- 1 root root  19K Aug 26 18:10 100017.json
-rw-r--r-- 1 root root  32K Aug 25 12:44 10001.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100022.json


In [10]:
!ls -1 themoviedb_data/movies | wc -l

409791


##ACCESO A MI DRIVE

In [11]:
#Esto me va a permitir tener acceso a las carpetas de mi drive desde Colab:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/" -lh

total 4.4G
drwx------ 2 root root  4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root   76K Oct 22 18:13  6885.json
drwx------ 2 root root  4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root  4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root   44K Feb  4  2018  contacts.vcf
-rw------- 1 root root  170M Oct 22 12:05  dfMoviesCast.csv
-rw------- 1 root root     1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root  1.5M Oct 24 17:34  IMDB_5000_movie.csv
-rw------- 1 root root     1 Oct 26 15:05 'Mis cuentas.gsheet'
-rw------- 1 root root 1019M Oct 14 16:41  movies.csv
-rw------- 1 root root  1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root     1 Oct 20 08:40 'Nuestras Cuentas.gsheet'
-rw------- 1 root root  1.8G Sep 24 21:07  people.zip
-rw------- 1 root root     1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root  4.0K Apr  7  2015  Screencastify
-rw------- 1 root root     1 Sep 30  2017  Series.gdoc
-rw----

## ESTE BLOQUE CONTIENE EL CÓDIGO QUE HE DE EJECUTAR PARA ESTRUCTURAR MIS DATOS:

EL DETALLE DE CÓMO HE LLEGADO A ESTE CÓDIGO ESTÁ EN EL NOTEBOOK 03_TIDYING_MOVIES_DATA.IPYNB

In [7]:
#Lo importo todo. Tengo que reducir el número de campos para que me quepa en memoria.
#Excluyo: backdrop_path, external_ids, homepage, images_backdrops, images_posters, poster_path, similar_movies_page/results/total_pages/total_results,
#video, recommendations_page/results/total_pages/total_results
#Recommendations: películas recomendadas para una película. No es lo mismo que similar_movies.
#Para hacer esto debería hacer una importación inicial de todos los campos y luego, una vez decidido, importo todo.

#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Poner la nota: importo todo, aunque sé que lo voy a usar, por si alguien puede querer usarlo.
#Desanido el primer nivel de diccionarios. Para belongs_to_collection, que a veces es nulo en lugar de un diccionario, tomo dos keys.
#Me queda tratar las listas que contienen diccionarios. 
#¿Qué pasa con UTF8? !!!!!!!!!!!!!!!!!!!!!!!!!!
#No vale de nada cambiar el orden!!! Así que mantengo el alfabético del csv
#Por mi código anterior ya sé que adult es un booleano y que no tiene nulos. Así que puedo filtrarlo por adult = False. Voy a excluir estas
#pelis de mi análisis.

#Sobre mi notebook 03 he incluido la variable imdb_id, que necesito para el cruce posterior que hago.

path = "./themoviedb_data/movies"
dfMovies = pd.DataFrame()

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if intI <= 500000:
                if f.endswith('.json'):
                    fp = os.path.join(root,f)
                    with open(fp) as o:
                        data = ujson.load(o)
                
                    if intI % 10000 == 0:
                        print(intI)
                    
                    result = {"titles": data["alternative_titles"]["titles"], 
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "genres": data["genres"], "id": data["id"], "imdb_id": data["imdb_id"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], 
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], 
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
                    if data["belongs_to_collection"] is None:
                        result["belongs_to_collection_id"] = None
                        result["belongs_to_collection_name"] = None
                    else:
                        result["belongs_to_collection_id"] = data["belongs_to_collection"]["id"]
                        result["belongs_to_collection_name"] = data["belongs_to_collection"]["name"]
                    
                    if data['adult'] == False:
                        yield result
            else:
                break          

tic=timeit.default_timer()
dfMovies = pd.DataFrame(data_iterator(path))
dfMovies.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


299.4319417379993

In [0]:
#columnName: columna de dfMovies a tratar. Sus elemeentos son listas de diccionarios.
#args: lista variable de claves con las que me voy a quedar de los diccionarios.
#Si un elemento es una lista vacía, no hay problema.
def columnIterator(columnName, *args):
    for id in dfMovies.index:
        for dict in dfMovies[dfMovies.index == id][columnName].iloc[0]:
            outputDict = {'movie_id': id}
            for key in args:
                outputDict[key] = dict[key]
            yield outputDict

In [0]:
#Creo mi dataframe con la info de la columna correspondiente, compruebo que tiene el nº filas que ha de tener y, si es así, elimino la columna
#correspondiente en dfMovies. Haré todo esto con la siguiente función:
def columnProcesses (columnName, *args):
    #I need to declare dfMovies as global as I will change it inside this function.
    global dfMovies
    
    dfOutput = pd.DataFrame()
    dfOutput = pd.DataFrame(columnIterator(columnName, *args))
    
    #Compruebo que tiene el nº de registros que ha de tener.
    dfCheck = pd.DataFrame(dfMovies[columnName].map(len).value_counts())
    dfCheck['calculation']=dfCheck[columnName]*dfCheck.index
    
    if dfOutput.shape[0] == dfCheck['calculation'].sum():
        dfMovies = dfMovies.drop(columnName, axis=1)
        return dfOutput
    else:
        return "Error in process"

In [0]:
def columnIteratorReleaseDates(*args):
    for id in dfMoviesReleaseDates1.index:
        for dict in dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['release_dates'].iloc[0]:
            outputDict = {'movie_id': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['movie_id'].iloc[0],
                         'iso_3166_1': dfMoviesReleaseDates1[dfMoviesReleaseDates1.index == id]['iso_3166_1'].iloc[0]}
            for key in args:
                outputDict[key] = dict[key]
            yield outputDict

In [0]:
#Generación de todas las tablas de una tacada: 
#This module took about 70min to be executed in Google Colab.
tic=timeit.default_timer()

dfMoviesCast = columnProcesses ('credits_cast', 'cast_id', 'character', 'gender', 'id', 'name', 'order')
print("credits_cast generated")
dfMoviesCrew = columnProcesses ('credits_crew', 'department', 'gender', 'id', 'job', 'name')
print("credits_crew generated")
dfMoviesGenres = columnProcesses ('genres', 'id', 'name')
print("genres generated")
dfMoviesKeywords = columnProcesses ('keywords', 'id', 'name')
print("keywords generated")
dfMoviesProductionComp = columnProcesses ('production_companies', 'id', 'name', 'origin_country')
print("production_companies generated")
dfMoviesProductionCountries = columnProcesses ('production_countries', 'iso_3166_1', 'name')
print("production_countries generated")
dfMoviesReviewsResults = columnProcesses ('reviews_results', 'author', 'content', 'id')
print("reviews_results generated")
dfMoviesSpokenLang = columnProcesses ('spoken_languages', 'iso_639_1', 'name')
print("spoken_languages generated")
dfMoviesReleaseDates1 = columnProcesses ('release_dates', 'iso_3166_1', 'release_dates')
print("release_dates 1 generated")
dfMoviesReleaseDates = pd.DataFrame(columnIteratorReleaseDates('certification', 'iso_639_1', 'release_date', 'type'))
print("release_dates generated")
#Libero la memoria
dfMoviesReleaseDates1 = None

toc=timeit.default_timer()
toc - tic #elapsed time in seconds

In [15]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_cast', 'credits_crew', 'genres', 'imdb_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

##EXPLORATORY DATA ANALYSIS FOR MOVIES

In [13]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_cast', 'credits_crew', 'genres', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

In [0]:
dfMovies.sample(3)

####Intento hacerlo con altair pero me encuentro con que hay un límite para 5mil registros.¿Quizá puedo primero agregar antes de representar?

In [0]:
#Intento hacerlo con altair pero me encuentro con que hay un límite para 5mil registros.
#¿Quizá puedo primero agregar antes de representar?

In [0]:
!pip install -U altair vega_datasets notebook vega

In [0]:
import altair as alt
from vega_datasets import data

In [0]:
#Si estoy en un notebook normal, ejecutaría:
#alt.renderers.enable("notebook")

#Pero en colab, ejecuto:
alt.renderers.enable("colab")

In [0]:
#Y aquí tengo el error.
alt.Chart(dfMovies).mark_bar().encode(
   #Como quiero hacer una modificación de x, tengo que incluir alt.X. Si fuese de y, sería alt.Y 
   x=alt.X("status", bin=True),
   y="count()" 
)

####Así que de momento lo hago con matplotlib o seaborn

In [0]:
#Si uso el código de Toni:
import matplotlib.pyplot as plt
%matplotlib inline  
#Para que nos haga los plot en la misma celda y que no salga una nueva pantalla
plt.style.use('ggplot')   
#Esto es para el estilo, pero da un poco igual

In [0]:
#Si uso seaborn, de los cursos de DataCamp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
sns.set()
#Esto es para el estilo, pero da un poco igual

####EXPLORACIÓN INICIAL:

In [16]:
#Veo qué columnas tienen algún nulo:
dfMovies.isnull().any()

belongs_to_collection_id       True
belongs_to_collection_name     True
budget                        False
credits_cast                  False
credits_crew                  False
genres                        False
imdb_id                        True
keywords                      False
original_language             False
original_title                False
overview                      False
popularity                    False
production_companies          False
production_countries          False
release_date                  False
release_dates                 False
revenue                       False
reviews_page                  False
reviews_results               False
reviews_total_pages           False
reviews_total_results         False
runtime                        True
spoken_languages              False
status                        False
tagline                       False
title                         False
titles                        False
vote_average                

In [17]:
dfMovies['status'].value_counts()

Released           387266
Post Production      2161
In Production        1262
Rumored              1230
Planned              1203
Canceled              117
Name: status, dtype: int64

In [18]:
#No entiendo cómo tienen ingresos las películas que no han sido "Released"
#Como revenue no tiene nulos, puedo contarlo para conocer el nº de registros.
dfMovies.groupby('status')['revenue'].agg(['sum','count'])

,sum,count
status,,
Canceled,186708860,117
In Production,1514636,1262
Planned,1,1203
Post Production,31336004,2161
Released,7395865752226,387266
Rumored,50653894,1230


In [0]:
#Debería quedarme en mi modelo sólo con las "Released". Podría después aplicarlo al resto.

In [21]:
#Me interesa ver de las pelis de antes cuántas tienen revenue!=0 (para ver si son solo unas pocas pelis las que producen ingreso cuando status!=Released)
dfMovies[dfMovies['revenue']!=0].groupby('status')['revenue'].agg(['sum','count'])

,sum,count
status,,
Canceled,186708860,1
In Production,1514636,7
Planned,1,1
Post Production,31336004,9
Released,7395865752226,9986
Rumored,50653894,11


In [22]:
dfMovies.shape[0]

393239

In [23]:
dfMovies[dfMovies['revenue']!=0].shape[0]

10015

In [24]:
dfMovies[dfMovies['revenue']>50].shape[0]

9654

In [25]:
#En principio, registros que tendría para mi modelo. ¿Excluiría las pelis con ingresos >0 y <50? Habría que echar un vistazo.
#¿Excluiría las pelis de este año? Porque quizás los datos de ingresos no se han estabilizado.
dfMovies[dfMovies['revenue']>50][dfMovies['status']=='Released'].shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


9628

In [26]:
#¿De cuántas pelis tengo el presupuesto?
dfMovies[dfMovies['budget']>50][dfMovies['status']=='Released'].shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


16937

In [27]:
#¿De cuántas pelis coincide que tengo presupuesto e ingresos?
dfMovies[dfMovies['revenue']>50][dfMovies['budget']>50][dfMovies['status']=='Released'].shape[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


6556

In [28]:
#Relajando el criterio de >50:
dfMovies[(dfMovies['revenue']>0) & (dfMovies['budget']>0) & (dfMovies['status']=='Released')].shape[0]

6823

In [29]:
dfMovies[dfMovies['title']=='Avatar'][['release_date','credits_cast']]

,release_date,credits_cast
id,,
421403,1941-09-16,"[{'cast_id': 1, 'character': 'Judge', 'credit_..."
19995,2009-12-10,"[{'cast_id': 242, 'character': 'Jake Sully', '..."
282908,2011-04-30,"[{'cast_id': 0, 'character': 'Michiko Abukumag..."


####VOY A TRAERME OTRO DATASET DE 5000 PELIS DE IMDB PARA MEJORAR LA INFO DE BUDGET Y REVENUE.

In [0]:
#¿Con qué criterio estará la info en este dataset?!!!!

In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#5000 movies from IMDB.
#Tuve que volver a ejecutar el código de las librerías de Google, porque si no ésta da error. Ejecuntando la anterior, veo que funciona.
#¿Puedo eliminar alguna línea del código de la celda anterior esta vez?

file_id = '1ebj6EkGylXTGldfGfJO_AHEAncNf4zKA'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("IMDB_5000_movie.csv")


In [13]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Oct 26 15:33 adc.json
drwx------ 3 root root 4.0K Oct 26 15:38 drive
-rw-r--r-- 1 root root 1.5M Oct 26 15:46 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 1.5G Oct 26 15:33 movies.zip
drwxr-xr-x 2 root root 4.0K Oct 23 16:44 sample_data
drwxr-xr-x 3 root root 4.0K Oct 26 15:34 themoviedb_data


In [14]:
!head IMDB_5000_movie.csv

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar ,886204,4834,Wes Studi,0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1,3054,English,USA,PG-13,237000000,2009,936,7.9,1.78,33000
Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End ,471220,48350,Jack Davenport,0,goddess|marriage ceremony|marriage proposal|pirate|singapore,http://www.imdb.com/title/tt0449088/?ref_=fn_tt_tt_1,

In [15]:
my_file_path='./IMDB_5000_movie.csv'

#Reading file and saving in df
dfMovies5000 = pd.read_csv(my_file_path)
dfMovies5000.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [23]:
#Miro el final a ver si hay algo raro:
dfMovies5000.tail()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660
5042,Color,Jon Gunn,43.0,90.0,16.0,16.0,Brian Herzlinger,86.0,85222.0,Documentary,...,84.0,English,USA,PG,1100.0,2004.0,23.0,6.6,1.85,456


In [19]:
dfMovies5000.shape

(5043, 28)

In [20]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [21]:
#De aquí tengo que extraer la clave de imdb para relacionarla con dfMovies.
dfMovies5000['movie_imdb_link'].sample(5)

4655    http://www.imdb.com/title/tt0295725/?ref_=fn_t...
1642    http://www.imdb.com/title/tt0121164/?ref_=fn_t...
912     http://www.imdb.com/title/tt1790885/?ref_=fn_t...
1319    http://www.imdb.com/title/tt0486822/?ref_=fn_t...
2096    http://www.imdb.com/title/tt0108101/?ref_=fn_t...
Name: movie_imdb_link, dtype: object

In [22]:
#Voy a ver primero cuántos nulos tengo por campo. ¿Tiene nulos movie_imdb_link?
dfMovies5000.isnull().any()
#Hay muchas columnas con nulos, pero movie_imdb_link no tiene ninguno.

color                         True
director_name                 True
num_critic_for_reviews        True
duration                      True
director_facebook_likes       True
actor_3_facebook_likes        True
actor_2_name                  True
actor_1_facebook_likes        True
gross                         True
genres                       False
actor_1_name                  True
movie_title                  False
num_voted_users              False
cast_total_facebook_likes    False
actor_3_name                  True
facenumber_in_poster          True
plot_keywords                 True
movie_imdb_link              False
num_user_for_reviews          True
language                      True
country                       True
content_rating                True
budget                        True
title_year                    True
actor_2_facebook_likes        True
imdb_score                   False
aspect_ratio                  True
movie_facebook_likes         False
dtype: bool

In [24]:
#Voy ahora a tratar de extraer el id de IMDB de movie_imdb_link que, como no tiene nulos, podría ser la clave.
dfMovies5000['movie_imdb_link'].sample(5)

449     http://www.imdb.com/title/tt2191701/?ref_=fn_t...
1197    http://www.imdb.com/title/tt1285016/?ref_=fn_t...
3905    http://www.imdb.com/title/tt2034139/?ref_=fn_t...
665     http://www.imdb.com/title/tt0106582/?ref_=fn_t...
2860    http://www.imdb.com/title/tt0269341/?ref_=fn_t...
Name: movie_imdb_link, dtype: object

In [25]:
dfMovies5000['movie_imdb_link'].sample(5).str.split("/").map(lambda list: list[4])

1577    tt0079574
161     tt0145487
1651    tt0420223
3071    tt1132626
4548    tt0497116
Name: movie_imdb_link, dtype: object

In [0]:
#Construyo una nueva columna con el identificador de imdb: id_imdb
dfMovies5000['imdb_id'] = dfMovies5000['movie_imdb_link'].str.split("/").map(lambda list: list[4])

In [17]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [28]:
dfMovies5000[['imdb_id', 'movie_imdb_link']].sample(5)

,imdb_id,movie_imdb_link
3144,tt0100168,http://www.imdb.com/title/tt0100168/?ref_=fn_t...
4193,tt0384504,http://www.imdb.com/title/tt0384504/?ref_=fn_t...
1091,tt0264472,http://www.imdb.com/title/tt0264472/?ref_=fn_t...
1588,tt0114709,http://www.imdb.com/title/tt0114709/?ref_=fn_t...
3678,tt0330602,http://www.imdb.com/title/tt0330602/?ref_=fn_t...


In [49]:
#Voy a asegurarme de que no tengo cadenas de longitud vacías en mi identificador. Así que, quitando blancos, me aseguro de que la longitud no es 0 y,
#en cambio, es siempre 9:
dfMovies5000['imdb_id'].str.strip().map(len).value_counts()

9    4919
Name: imdb_id, dtype: int64

In [29]:
#¿Tengo duplicados?
#(en dfMovies ya lo sé, por cómo descargué la info)
dfMovies5000['imdb_id'].duplicated().sum()
#Hay 124 claves que aparecen más de una vez (sin contar las duplicidades)

124

In [30]:
#Echo un vistazo a algunos:
dfMovies5000['imdb_id'][dfMovies5000['imdb_id'].duplicated()].head()
#Esto no me da los dos (o más) registros con igual clave imdb_id duplicada, sino sólo uno de ellos.

137    tt0918940
187    tt1673434
204    tt4262980
303    tt3332064
389    tt1502712
Name: imdb_id, dtype: object

In [31]:
dfMovies5000[dfMovies5000['imdb_id'] == 'tt0918940']

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,imdb_id
63,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,English,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000,tt0918940
137,Color,David Yates,248.0,110.0,282.0,103.0,Alexander Skarsgård,11000.0,124051759.0,Action|Adventure|Drama|Romance,...,English,USA,PG-13,180000000.0,2016.0,10000.0,6.6,2.35,29000,tt0918940


In [32]:
#Aunque hay 124 códigos imdb_id repetidos, sólo 45 registros registros están repetidos para todos los campos.
dfMovies5000.duplicated().sum()

45

In [33]:
#Y a nivel de imdb_id, gross y budget tengo 123 duplicados:
#REalmente, de momento, sólo voy a tomar la info de gross y budget (¿me podría interesar la de los likes???)
dfMovies5000[['imdb_id', 'gross', 'budget']].duplicated().sum()

123

In [0]:
#O sea: los duplicados a nivel de clave lo son también a nivel de gross y budget salvo por un registro. Voy a eliminar los
#duplicados a nivel de clave quedándome con el primero de los registros que me encuentre (SIMPLIFICACIÓN).

In [0]:
dfMovies5000 = dfMovies5000.drop_duplicates(subset='imdb_id', keep='first')

In [19]:
dfMovies5000.shape

(4919, 29)

In [40]:
#Y ahora ya puedo cruzar con dfMovies
#Voy a ver primero de cuántas pelis tengo budget y "gross", que son las variables que seguro voy a usar. No tengo claro que vaya a usar el resto.
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].sample(5)

,movie_imdb_link,gross,budget
2685,http://www.imdb.com/title/tt0475998/?ref_=fn_t...,12701880.0,15000000.0
336,http://www.imdb.com/title/tt1661199/?ref_=fn_t...,201148159.0,95000000.0
2814,http://www.imdb.com/title/tt1460743/?ref_=fn_t...,NaN,12000000.0
3476,http://www.imdb.com/title/tt1343092/?ref_=fn_t...,144812796.0,105000000.0
3123,http://www.imdb.com/title/tt0098206/?ref_=fn_t...,30050028.0,17000000.0


In [38]:
#Ya vimos que gross y budget tienen nulos.
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].isnull().any()

movie_imdb_link    False
gross               True
budget              True
dtype: bool

In [39]:
dfMovies5000[['movie_imdb_link', 'gross', 'budget']].count()

movie_imdb_link    4919
gross              4056
budget             4434
dtype: int64

In [0]:
#Cuando vaya a actualizar, estaría bien tener una marca del origen del dato. !!!!!!!!!!!!!!!!!!!!!!!

In [40]:
#Bueno, parece que aunque hay nulos no hay 0's.
dfMovies5000[(dfMovies5000['gross'].notnull()) & (dfMovies5000['gross']>0)].shape

(4056, 29)

In [41]:
#¿Y si exigo que los beneficios sean más de 50$? Bien: no cambia el número. Me da indicio de que el dato es más fiable que el de TMDB.
dfMovies5000[(dfMovies5000['gross'].notnull()) & (dfMovies5000['gross']>50)].shape

(4056, 29)

In [42]:
#Echo un vistazo de igual modo al budget: estupendo, no hay 0's
dfMovies5000[(dfMovies5000['budget'].notnull()) & (dfMovies5000['budget']>0)].shape

(4434, 29)

In [43]:
#Ni tampoco pelis con presupuesto menor o igual a 50$
dfMovies5000[(dfMovies5000['budget'].notnull()) & (dfMovies5000['budget']>50)].shape

(4434, 29)

In [44]:
#Tendré que homogeneizar criterios con dfMovies: no hay nulos pero sí 0's en revenue y budget.

#Otra cosa a tener en cuenta: no todas las pelis en dfMovies tenían identificador de imdb. Podría hacerme perder algunos registros en el cruce.
#Poco puedo hacer. No voy a anexar la info de lo que no cruce. Lo despreciaré.
dfMovies[['imdb_id', 'status']].count()

imdb_id    337380
status     393239
dtype: int64

In [45]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [0]:
#Aquí no tengo el id de actor para cruzar con dfMovies. Así que voy a pasar de las variables "actor_1_facebook_likes", "actor_2_facebook_likes",
#"actor_3_facebook_likes", pero me voy a quedar con director_facebook_likes, movie_facebook_likes y "cast_total_facebook_likes". 
#Esta última, ¿suma las anteriores o es realmente la suma total de todo el cast?

In [51]:
#Esto no debería devolver registros:
dfMovies5000[dfMovies5000['actor_1_facebook_likes']+dfMovies5000['actor_2_facebook_likes']+dfMovies5000['actor_3_facebook_likes']>dfMovies5000['cast_total_facebook_likes']]
#Guay, no los devuelve.

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,imdb_id


In [53]:
#¿La suma total de likes de todo el reparto suele ser mayor que la suma de likes de los 3 primers actores? Sí, suele serlo!
dfMovies5000[dfMovies5000['actor_1_facebook_likes']+dfMovies5000['actor_2_facebook_likes']+dfMovies5000['actor_3_facebook_likes']<dfMovies5000['cast_total_facebook_likes']].shape

(4746, 29)

In [61]:
#¿De cuántos años tengo pelis? De un montón!!
dfMovies5000['title_year'].value_counts().sort_index()
#Que equivale a:
#dfMovies5000.groupby('title_year')['imdb_id'].count()

1916.0      1
1920.0      1
1925.0      1
1927.0      1
1929.0      2
1930.0      1
1932.0      1
1933.0      2
1934.0      1
1935.0      1
1936.0      2
1937.0      2
1938.0      2
1939.0      3
1940.0      5
1941.0      1
1942.0      2
1943.0      1
1944.0      1
1945.0      4
1946.0      3
1947.0      3
1948.0      3
1949.0      2
1950.0      1
1951.0      3
1952.0      4
1953.0      4
1954.0      4
1955.0      2
         ... 
1987.0     32
1988.0     30
1989.0     32
1990.0     29
1991.0     31
1992.0     34
1993.0     48
1994.0     54
1995.0     70
1996.0     98
1997.0    117
1998.0    132
1999.0    167
2000.0    169
2001.0    183
2002.0    204
2003.0    169
2004.0    207
2005.0    216
2006.0    235
2007.0    198
2008.0    223
2009.0    253
2010.0    225
2011.0    224
2012.0    214
2013.0    231
2014.0    243
2015.0    211
2016.0     98
Name: title_year, Length: 91, dtype: int64

In [63]:
#¿De cuántos países tengo pelis? Muy sesgado al cine en inglés (aunque tengo pelis de otros países)
dfMovies5000['country'].value_counts().sort_values(ascending=False)

USA                     3711
UK                       434
France                   154
Canada                   124
Germany                   94
Australia                 53
India                     34
Spain                     33
China                     28
Italy                     23
Japan                     22
Hong Kong                 17
Mexico                    17
South Korea               14
New Zealand               13
Ireland                   12
Russia                    11
Denmark                   11
South Africa               8
Brazil                     8
Norway                     8
Sweden                     6
Netherlands                5
Thailand                   5
Israel                     4
Poland                     4
Iran                       4
Argentina                  4
Romania                    4
West Germany               3
                        ... 
Hungary                    2
Taiwan                     2
Official site              1
Cambodia      

In [64]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

####CRUCE CON LA INFO DE TMDB:

In [20]:
#Voy a cruzar con dfMovies. 
#¿Tengo duplicados en el index? No me hace falta para el cruce pero debería haberlo visto ya:
dfMovies.index.duplicated().sum()

0

In [21]:
#Recuerdo que tengo nulos en dfMovies en imdb_id:
dfMovies[['imdb_id', 'status']].isnull().any()

imdb_id     True
status     False
dtype: bool

In [43]:
#¿Cuántos nulos?
dfMovies[['imdb_id', 'status']].isnull().sum()

imdb_id    55859
status         0
dtype: int64

In [42]:
#Resulta que además de nulos, hay valores ''
dfMovies[dfMovies['imdb_id']==''].shape[0]

45838

In [36]:
#Los veo aquí, donde además veo un valor que aparece dos veces: tt1564005
dfMovies['imdb_id'].value_counts()[0:5]

             45838
tt1564005        2
tt1160484        1
tt2514646        1
tt0108662        1
Name: imdb_id, dtype: int64

In [39]:
#Excluyendo los nulos y los '', parece que mi único duplicado es ese valor:
dfMovies[(dfMovies['imdb_id'].notnull()) & (dfMovies['imdb_id']!='')]['imdb_id'].duplicated().sum()

1

In [0]:
#Debería sustituir las cadenas de longitud vacía por nulos, para tener un criterio homogéneo. Lo hago:
#Podría mejorarlo (si necesito aplicarlo a otros) y convertir cualquier cadena de longitud vacía en nulos.
#Esta función se aplica a cualquier tipo de dato y devuelvo un None si la entrada es ''
def fill_blank_with_nones(value):
    if value=='':
        return None
    else:
        return value

In [0]:
dfMovies['imdb_id']=dfMovies['imdb_id'].map(fill_blank_with_nones)

In [91]:
#Veo si lo he hecho bien: ¿cuántos nulos tengo ahora? Perfecto!
dfMovies['imdb_id'].isnull().sum()

101697

In [46]:
#Voy a echar un vistazo al duplicado:
dfMovies[dfMovies['imdb_id']=='tt1564005']
#Realmente, se refiere a la misma peli. Aunque algún campo es diferente. Lo dejo estar de momento. Tampoco sé seguro que sea un error.

,belongs_to_collection_id,belongs_to_collection_name,budget,credits_cast,credits_crew,genres,imdb_id,keywords,original_language,original_title,...,reviews_total_pages,reviews_total_results,runtime,spoken_languages,status,tagline,title,titles,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,
101318,NaN,None,0,"[{'cast_id': 1004, 'character': 'Joss Jenner',...","[{'credit_id': '52fe49efc3a36847f81ae499', 'de...","[{'id': 18, 'name': 'Drama'}]",tt1564005,[],en,Lies Between Friends,...,0,0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Lies Between Friends,"[{'iso_3166_1': 'ES', 'title': 'Asesinato entr...",4.8,4
545015,NaN,None,0,"[{'cast_id': 0, 'character': 'Joss Jenner', 'c...",[],"[{'id': 9648, 'name': 'Mystery'}, {'id': 10770...",tt1564005,[],en,Lies Between Friends,...,0,0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Lies Between Friends,[],5.0,1


In [93]:
#Voy a cruzar. Antes controlo cuántos cruzan:
dfMovies.merge(dfMovies5000, how='inner', left_on='imdb_id', right_on='imdb_id').shape

(4798, 57)

In [94]:
#Estupendo: cruzan la mayoría.
dfMovies5000.shape

(4919, 29)

In [96]:
dfMovies.merge(dfMovies5000, how='inner', left_on='imdb_id', right_on='imdb_id').columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget_x',
       'credits_cast', 'credits_crew', 'genres_x', 'imdb_id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count', 'color', 'director_name',
       'num_critic_for_reviews', 'duration', 'director_facebook_likes',
       'actor_3_facebook_likes', 'actor_2_name', 'actor_1_facebook_likes',
       'gross', 'genres_y', 'actor_1_name', 'movie_title', 'num_voted_users',
       'cast_total_facebook_likes', 'actor_3_name', 'facenumber_in_poster',
       'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews', 'language',
       'country', 'content_rating', 'budget_y', 'title_year'

In [98]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [0]:
#Quiero cambiar el nombre de algunas columnas antes de cruzar, para que sean más fácilmente identificables:
def change_column_names(str):
    if str == 'budget':
        return 'imdb_budget'
    elif str == 'gross':
        return 'imdb_revenue'
    else:
        return str

In [0]:
dfMovies5000.columns = list(map(change_column_names, dfMovies5000.columns.tolist()))

In [108]:
dfMovies5000.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'imdb_revenue', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'imdb_budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes', 'imdb_id'],
      dtype='object')

In [0]:
#Hago el cruce:
dfMovies = dfMovies.merge(dfMovies5000[['imdb_id', 'imdb_revenue', 'imdb_budget', 'director_facebook_likes', 'cast_total_facebook_likes', 'movie_facebook_likes']], how='left', left_on='imdb_id', right_on='imdb_id')

In [113]:
dfMovies.shape

(393239, 34)

In [118]:
#Pero ahora tengo distinto criterio en budget y revenue para tmdb e imdb:
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].isnull().sum()

budget               0
revenue              0
imdb_budget     388825
imdb_revenue    389188
dtype: int64

In [119]:
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].notnull().sum()

budget          393239
revenue         393239
imdb_budget       4414
imdb_revenue      4051
dtype: int64

In [0]:
#Igualo criterios:
def change_none_to_zero(num):
    if num != num or num is None:
        return 0
    else:
        return num

In [0]:
lista = [None, 3, 'casa', np.NaN]

In [137]:
list(map(change_none_to_zero, lista))

[0, 3, 'casa', 0]

In [0]:
dfMovies['imdb_budget'] = dfMovies['imdb_budget'].map(change_none_to_zero)
dfMovies['imdb_revenue'] = dfMovies['imdb_revenue'].map(change_none_to_zero)

In [140]:
#Chequeo:
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].notnull().sum()

budget          393239
revenue         393239
imdb_budget     393239
imdb_revenue    393239
dtype: int64

In [142]:
#¿Cuántos 0s tengo?
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].applymap(lambda num: num==0).sum()

budget          374718
revenue         383224
imdb_budget     388825
imdb_revenue    389188
dtype: int64

In [143]:
#¿Cuántos valores distintos de 0 tengo?
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue']].applymap(lambda num: num!=0).sum()

budget          18521
revenue         10015
imdb_budget      4414
imdb_revenue     4051
dtype: int64

In [144]:
#Poniendo el foco en revenue, ¿en cuántos casos tengo dato distinto de 0 en imdb_revenue pero igual a 0 en revenue?
dfMovies[(dfMovies['revenue']==0) & (dfMovies['imdb_revenue']>0)].shape[0]

842

In [148]:
#¿Y en cuántos casos tengo dato distinto de 0 en imdb_revenue pero menor o igual que 50 en revenue? Recordar: imdb_revenue era siempre >50
dfMovies[(dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']>0)].shape[0]

858

In [146]:
#Hago un ejercicio similar para budget:
dfMovies[(dfMovies['budget']==0) & (dfMovies['imdb_budget']>0)].shape[0]

680

In [149]:
dfMovies[(dfMovies['budget']<=50) & (dfMovies['imdb_budget']>0)].shape[0]

705

In [0]:
#Voy a adoptar un criterio simple ahora: tomo el dato de tmdb de revenue o budget, a no ser que sea 0 o <50, en cuyo caso tomo el de imdb.
#Afinarlo más adelante!!!!
dfMovies['revenue_final'] = dfMovies['revenue'] * ((dfMovies['revenue']>50) | ((dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']==0))) + dfMovies['imdb_revenue'] * (~((dfMovies['revenue']>50) | ((dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']==0))))
dfMovies['budget_final'] = dfMovies['budget'] * ((dfMovies['budget']>50) | ((dfMovies['budget']<=50) & (dfMovies['imdb_budget']==0))) + dfMovies['imdb_budget'] * (~((dfMovies['budget']>50) | ((dfMovies['budget']<=50) & (dfMovies['imdb_budget']==0))))

In [161]:
#Chequeo que lo he hecho bien:
dfMovies[(dfMovies['revenue']<=50) & (dfMovies['imdb_revenue']>0)][['revenue', 'imdb_revenue', 'revenue_final']].head()

,revenue,imdb_revenue,revenue_final
926,0,33422806.0,33422806.0
1169,0,267194.0,267194.0
1947,0,3064356.0,3064356.0
2037,0,10443.0,10443.0
2052,0,105943.0,105943.0


In [162]:
dfMovies[(dfMovies['revenue']>50) & (dfMovies['imdb_revenue']>0)][['revenue', 'imdb_revenue', 'revenue_final']].head()

,revenue,imdb_revenue,revenue_final
39,21126225,6301131.0,21126225.0
208,120000,58214.0,120000.0
284,150277,145540.0,150277.0
483,88923251,59573085.0,88923251.0
524,93525586,43982842.0,93525586.0


In [163]:
dfMovies[(dfMovies['revenue']<=50) & (dfMovies['revenue']>0) & (dfMovies['imdb_revenue']>0)][['revenue', 'imdb_revenue', 'revenue_final']].head()

,revenue,imdb_revenue,revenue_final
47080,7,7680.0,7680.0
55071,10,10149779.0,10149779.0
71371,25,23807.0,23807.0
73488,15,13350177.0,13350177.0
88488,12,12189514.0,12189514.0


In [164]:
#¿Cuántos valores distintos de 0 tengo?
dfMovies[['budget', 'revenue', 'imdb_budget', 'imdb_revenue', 'budget_final', 'revenue_final']].applymap(lambda num: num!=0).sum()

budget           18521
revenue          10015
imdb_budget       4414
imdb_revenue      4051
budget_final     19201
revenue_final    10857
dtype: int64

In [165]:
#Para saber el uso de memoria que estoy haciendo:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

7706472448


In [0]:
#Para eliminar de la memoria
#dfMovies5000Cleaned = pd.DataFrame()
#del dfMovies5000Cleaned